In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import random
import os
import pickle
from sklearn import metrics
from datetime import date
from sklearn.metrics import classification_report
import Evaluation_function_for_online as func
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

In [2]:
file_path = 'D:\\test_prediction_results_201911.csv'
origin_path = 'D:\\201911_label.csv'

prediction, df = func.read_prediction(file_path, origin_path)
df

,ts,StockNo,close,VWAP,foreign_ratio,investment_ratio,corporation_ratio,PctRank,index_close,industry_close,VWAP_day5,VWAP_after,Y,future_return,Y_0_score,Y_1_score
0,2019-11-01,1101,40.55,40.48,29.73,0.46,31.22,0.956150,11399.53,159.39,40.512664,40.848552,1.0,0.335889,0.580823,0.419177
1,2019-11-04,1101,40.95,40.90,29.79,0.46,31.29,0.956150,11556.85,160.79,40.565334,40.775078,1.0,0.209743,0.266631,0.733369
2,2019-11-05,1101,40.90,40.77,29.80,0.46,31.32,0.956150,11644.03,160.60,40.600743,40.747329,1.0,0.146585,0.380035,0.619965
3,2019-11-06,1101,40.80,40.84,29.81,0.46,31.35,0.956150,11653.07,159.93,40.712338,40.843054,1.0,0.130716,0.437605,0.562395
4,2019-11-07,1101,40.95,40.80,29.78,0.46,31.34,0.956150,11606.56,160.30,40.781419,40.897336,1.0,0.115917,0.464841,0.535159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19181,2019-11-25,9958,72.70,72.56,1.76,1.68,4.45,0.862032,11561.58,99.58,72.194779,72.333791,1.0,0.139012,0.415136,0.584864
19182,2019-11-26,9958,73.80,73.68,1.98,1.65,4.68,0.862032,11576.82,100.24,72.726391,71.696630,0.0,-1.029760,0.279482,0.720517
19183,2019-11-27,9958,73.10,73.35,1.86,1.65,4.54,0.862032,11647.46,100.38,72.926809,71.390299,0.0,-1.536509,0.464203,0.535797
19184,2019-11-28,9958,72.20,72.59,1.91,1.65,4.59,0.862032,11617.08,100.39,73.025855,71.196388,0.0,-1.829467,0.684399,0.315601


In [3]:
func.verification(df)

True

In [4]:
test_year = [2017, 2018, 2019]
threshold = [0.5, 0.55, 0.6, 0.65]

In [9]:
accuracy_list = []
for i in [j/10 for j in range(1, 11)]:
    accuracy_list.append(func.Evaluation(df[(df['PctRank'] >= i - 0.1) & (df['PctRank'] < i)], threshold=0.5)[0])

In [13]:
accuracy_list = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q:
    )

In [5]:
for t in threshold:
    r = func.Evaluation(df, threshold=t)
    print(f'The threshold is {t}')
    print(f'Accuracy: {r[0]} \n up_support/ratio: {r[2]}/{r[4]} \n down_support/ratio: {r[3]}/{r[5]}')
    print(r[1])

The threshold is 0.5
Accuracy: 0.6745022412175544 
 up_support/ratio: 7842/0.4087355363285729 
 down_support/ratio: 11344/0.591264463671427
              precision    recall  f1-score   support

        down       0.70      0.74      0.72     10807
          up       0.64      0.60      0.62      8379

    accuracy                           0.67     19186
   macro avg       0.67      0.67      0.67     19186
weighted avg       0.67      0.67      0.67     19186

The threshold is 0.55
Accuracy: 0.6984369275857858 
 up_support/ratio: 6453/0.33633899718544774 
 down_support/ratio: 9925/0.5173042843740228
              precision    recall  f1-score   support

        down       0.72      0.77      0.74      9304
          up       0.67      0.61      0.63      7074

    accuracy                           0.70     16378
   macro avg       0.69      0.69      0.69     16378
weighted avg       0.70      0.70      0.70     16378

The threshold is 0.6
Accuracy: 0.726479797226778 
 up_support/ra

In [5]:
df_list = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q.iloc[5:]:
    d = func.Stock_evaluation(df, threshold=t)
    display(d)
    df_list.append(d)



,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,foreign_ratio,investment_ratio,corporation_ratio
1102,0.9,0.762108,0.0178039,0.0177553,1,0.833333,8,12,28.0035,0.8855,29.71
1109,0.75,0.392496,0.0219935,0.0218086,0.909091,0.555556,11,9,2.7485,0.0145,2.773
1201,0.571429,0.354053,0.0138483,0.0137778,0.777778,0.416667,9,12,13.6971,0.110476,14.2724
1210,0.684211,0.29818,0.00799935,0.00795712,0.833333,0.615385,6,13,17.7568,0.738947,18.5463
1215,0.611111,0.542117,0.00846241,0.00840125,0.857143,0.454545,7,11,43.0011,1.87333,45.2283
...,...,...,...,...,...,...,...,...,...,...,...
6209,0.882353,0.569176,0.0197422,0.0196502,1,0.777778,8,9,2.79412,1.32,4.23235
6277,0.555556,0.849371,0.00999569,0.00984184,0.5,0.6,8,10,16.5122,0.07,16.7594
8016,0.705882,4.20558,0.0235364,0.0234574,0.9,0.428571,10,7,41.1724,9.03176,51.1665
8926,0.823529,0.406373,0.0140298,0.0139638,0.75,0.846154,4,13,7.03176,0.521176,8.34882


D:\庫存健診開發\code\PredictModel\Evaluation_function_for_online.py:281: RuntimeWarning:

invalid value encountered in double_scalars



,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,foreign_ratio,investment_ratio,corporation_ratio
1109,0.777778,0.439669,0.0246198,0.0245058,1,0.5,10,8,2.72944,0.0138889,2.75333
1201,0.5,0.286935,0.0112665,0.0111658,0.666667,0.4,6,10,13.6969,0.11125,14.2737
1210,0.6875,0.326419,0.00874404,0.00871243,1,0.583333,4,12,17.805,0.7425,18.5987
1215,0.615385,0.542117,0.00846241,0.00840125,0.857143,0.333333,7,6,43.0308,1.92846,45.3131
1216,0.928571,1.30098,0.0177738,0.0176967,1,0.857143,7,7,55.0064,0.639286,56.5879
...,...,...,...,...,...,...,...,...,...,...,...
3049,0.909091,0.229596,0.0160943,0.0160506,1,0.857143,4,7,10.2482,0.07,10.7255
5534,0.833333,1.23893,0.0154405,0.0154103,1,0.6,7,5,21.2792,0.79,22.5175
2030,0.444444,-0.0611511,-0.00473676,-0.00473796,0,0.571429,2,7,5.46889,0.17,5.67889
3033,0.666667,-0.14912,-0.0082041,-0.0082041,0,0.8,1,5,5.03833,0,5.31833


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,foreign_ratio,investment_ratio,corporation_ratio
1109,0.8,0.41738,0.0232664,0.0231562,1,0.571429,8,7,2.73533,0.014,2.75933
1201,0.583333,0.444202,0.017429,0.0174136,1,0.375,4,8,13.7142,0.113333,14.2883
1210,0.818182,0.416447,0.0111622,0.0111503,1,0.75,3,8,17.8382,0.749091,18.64
1215,0.5,0.691038,0.0108064,0.0107707,1,0,3,3,43.1083,2.01833,45.465
1216,0.916667,1.30098,0.0177738,0.0176967,1,0.8,7,5,54.975,0.639167,56.555
...,...,...,...,...,...,...,...,...,...,...,...
1457,0.8,0.158156,0.0112985,0.0112717,1,0,4,1,3.292,0.01,3.352
2707,0.857143,3.46382,0.0220012,0.022011,1,0.5,5,2,15.5771,0.184286,15.94
4104,1,0.320466,0.00602041,0.00602866,1,0,3,0,7.75,0.03,8.04
1701,0.5,0.0570432,0.0030213,0.00300782,0.5,0,2,0,10.28,0,10.31


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,foreign_ratio,investment_ratio,corporation_ratio
1201,0.714286,0.477655,0.0185989,0.0185444,1,0.6,2,5,13.7086,0.108571,14.2843
1216,1,1.62853,0.0223504,0.0223504,1,1,1,2,55.1333,0.646667,56.73
1304,0.8,-0.000625445,-4.88521e-05,-4.88521e-05,0,1,1,4,34.328,0.12,34.958
1305,0.9,1.46939,0.0722504,0.0716699,1,0.666667,7,3,14.789,0.964,15.834
1314,1,0.223664,0.0219423,0.0219408,1,1,2,4,18.5117,0.45,19.7817
...,...,...,...,...,...,...,...,...,...,...,...
2420,1,0.15997,0.00450823,0.00450823,1,0,1,0,4.18,0.37,4.63
2884,1,0.69997,0.0259375,0.0259375,1,0,1,0,44.46,1.2,45.69
3049,1,NaN,NaN,NaN,0,1,0,2,10.14,0.07,10.575
4942,1,2.34016,0.0602831,0.0602831,1,0,1,0,19.57,0.25,19.85


In [7]:
d1 = func.daily_evaluation(df, threshold=0.5)

In [8]:
for col in ['accuracy', 'up_precision', 'up_support', 'down_precision']:
    print(f'{col} min: {d1[col].min()}')
    print(f'{col} max: {d1[col].max()}')
    print(f'{col} mean: {d1[col].mean()}')
    print(f'{col} std: {d1[col].std()}')

accuracy min: 0.544069640914037
accuracy max: 0.7633587786259542
accuracy mean: 0.6726632545155893
accuracy std: 0.04313817493000802
up_precision min: 0.5150501672240803
up_precision max: 0.8440366972477065
up_precision mean: 0.72048574126707
up_precision std: 0.0879208614298518
up_support min: 228
up_support max: 693
up_support mean: 420.1904761904762
up_support std: 145.1091379092368
down_precision min: 0.4327956989247312
down_precision max: 0.7650085763293311
down_precision mean: 0.6054229924101469
down_precision std: 0.09202765603971355


In [5]:
df_list1 = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q.iloc[5:]:
    d = func.daily_evaluation(df, threshold=t)
    display(d)
    df_list1.append(d)

,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,index
2019-10-01,0.727463,2.42548,0.0133763,0.0228198,0.661654,0.752907,133,344,10967.6
2019-10-02,0.759754,2.26656,0.0168945,0.0201888,0.709302,0.787302,172,315,10947.9
2019-10-03,0.720648,3.13944,0.0281528,0.0352745,0.784314,0.704082,102,392,10875.9
2019-10-04,0.686192,2.67436,0.0260038,0.0304585,0.726027,0.668675,146,332,10894.5
2019-10-07,0.702306,1.7905,0.0219862,0.0246497,0.761682,0.653992,214,263,10935.1
2019-10-08,0.707627,1.77905,0.0241379,0.0250319,0.799065,0.631783,214,258,11017.3
2019-10-09,0.529532,3.82144,0.0427242,0.0437812,0.908257,0.421466,109,382,10890
2019-10-14,0.781186,1.67284,0.0281937,0.0203703,0.881967,0.61413,305,184,11067
2019-10-15,0.728916,1.42576,0.0275418,0.016836,0.868339,0.480447,319,179,11111.8
2019-10-16,0.796875,1.36395,0.0243204,0.0223701,0.877922,0.551181,385,127,11162.8


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,index
2019-10-01,0.766839,3.14271,0.0184959,0.0264109,0.718447,0.784452,103,283,10967.6
2019-10-02,0.797436,2.96249,0.0210639,0.0235647,0.782258,0.804511,124,266,10947.9
2019-10-03,0.758621,3.80504,0.0359278,0.0417924,0.855072,0.738872,69,337,10875.9
2019-10-04,0.738292,3.39144,0.0357217,0.0357055,0.835165,0.705882,91,272,10894.5
2019-10-07,0.746398,2.32242,0.0275434,0.0302288,0.797386,0.706186,153,194,10935.1
2019-10-08,0.75,2.38088,0.0320814,0.0316073,0.871622,0.66,148,200,11017.3
2019-10-09,0.543536,4.59675,0.0549631,0.0543692,0.919355,0.470032,62,317,10890
2019-10-14,0.817284,2.2033,0.0298783,0.0314983,0.903475,0.664384,259,146,11067
2019-10-15,0.78392,1.90008,0.0302255,0.0274083,0.886364,0.58209,264,134,11111.8
2019-10-16,0.834135,1.38662,0.0256533,0.0229331,0.883436,0.655556,326,90,11162.8


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,index
2019-10-01,0.817544,4.25309,0.0236562,0.0289169,0.783784,0.829384,74,211,10967.6
2019-10-02,0.835766,4.63229,0.0308024,0.0286048,0.831169,0.837563,77,197,10947.9
2019-10-03,0.780255,4.7886,0.0375892,0.0453581,0.886364,0.762963,44,270,10875.9
2019-10-04,0.767241,4.07751,0.0463906,0.0385013,0.833333,0.75,48,184,10894.5
2019-10-07,0.808824,3.92061,0.0407956,0.0450285,0.870588,0.764706,85,119,10935.1
2019-10-08,0.791111,3.51657,0.0427523,0.0436887,0.914634,0.72028,82,143,11017.3
2019-10-09,0.576271,5.47644,0.0723497,0.0748961,0.921053,0.525292,38,257,10890
2019-10-14,0.84,2.69616,0.0349194,0.0361669,0.927461,0.682243,193,107,11067
2019-10-15,0.846154,2.16747,0.0333874,0.0293679,0.894977,0.7125,219,80,11111.8
2019-10-16,0.854839,1.54109,0.0286703,0.0247159,0.886719,0.703704,256,54,11162.8


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support,down_support,index
2019-10-01,0.877419,6.67975,0.0346965,0.0327751,0.931818,0.855856,44,111,10967.6
2019-10-02,0.876812,6.21863,0.0472034,0.0513725,0.90625,0.867925,32,106,10947.9
2019-10-03,0.848958,5.95483,0.0494123,0.0567547,0.961538,0.831325,26,166,10875.9
2019-10-04,0.85,5.75913,0.0583897,0.0602927,0.913043,0.831169,23,77,10894.5
2019-10-07,0.842105,4.49563,0.043918,0.0472501,0.846154,0.839286,39,56,10935.1
2019-10-08,0.836735,5.92097,0.0705933,0.0630489,0.969697,0.769231,33,65,11017.3
2019-10-09,0.624309,8.63627,0.10131,0.0931502,0.888889,0.595092,18,163,10890
2019-10-14,0.890909,3.70387,0.0436123,0.0424612,0.936937,0.796296,111,54,11067
2019-10-15,0.915033,3.17769,0.0411606,0.0343211,0.93913,0.842105,115,38,11111.8
2019-10-16,0.900552,1.8619,0.0335245,0.0264442,0.917722,0.782609,158,23,11162.8


In [8]:
df_list1 = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q.iloc[5:]:
    d = func.monthly_evaluation(df, threshold=t)
    display(d)
    df_list1.append(d)

,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.719825,1.85493,0.0222658,0.0237212,0.812121,0.641772,220,260.143,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.758094,2.19179,0.025709,0.027697,0.850216,0.684186,165,205.667,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.796763,2.6944,0.0305408,0.0325784,0.886147,0.728029,110,143.048,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.853333,3.84143,0.0399737,0.0406051,0.925541,0.799353,55,73.5714,11358.7


In [9]:
func.monthly_evaluation(df, threshold=0.5)

,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.69645,1.69173,0.0204949,0.0217015,0.787334,0.618381,254.143,295.857,11358.7


In [15]:
for y in test_year:
    for df4 in df_list:
        d, r = func.daily_aggregate(df4, year=y)
        print(f'The threshold is {t}, year {y}')
        display(d)
        display(r['daily_accuracy_worst'])
        display(r['daily_ratio_worst'])
        display(r['daily_portfolio_worst'])

AttributeError: 'DataFrame' object has no attribute 'ts'

In [10]:
df_index = func.get_daily_index(date(2019,10,1), date(2019,10,31))
df_index = df_index.sort_values(by='ts').reset_index(drop=True)

In [11]:
df1 = df_list1[0].sort_index()
fig2 = make_subplots(rows=3, cols=1)

fig2.append_trace(
    go.Scatter(
        x=df1.index,
        y=df1['up_precision'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="precision",
        ),
    
    row=1, col=1,
)

fig2.append_trace(
    go.Scatter(
        x=df1.index,
        y=df1['up_support'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="up_count",
        ),
    
    row=2, col=1,
)

fig2.append_trace(
    go.Scatter(
        x=df_index['ts'],
        y=df_index['index_close'],
        mode="lines",
        line=go.scatter.Line(color="blue"),
        name="index",
        ),
    
    row=3, col=1,
)

# fig.update_layout(xaxis_rangeslider_visible=False, width=800)
# fig2.update_layout(width=835,height=300)
# fig2.update_yaxes(range=[0, 1])

fig2.show()
# fig.show()
